In [22]:
import os
import time

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd

import datasets
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoConfig, AutoModelForSequenceClassification, pipeline, BertForSequenceClassification

In [23]:
# Загрузка CSV в pandas DataFrame
import pandas as pd

test_df = pd.read_csv("test.csv")
print(test_df.shape)
test_df.head()

(2679, 3)


,text,label,id
0,все мы помним мой ахуй https://t.co/3Kr7yzeYLC,skip,1318021368932421633
1,я считаю это мем года https://t.co/xoVKj5y8Mj,positive,1218052288964632576
2,ян русский на сотку все запятые где надо🤙🏻👍🏻👍🏻...,positive,1212859589592539136
3,бэк дор соти https://t.co/bYCWz4h4Sk,skip,1306944400824709120
4,@daria_karapet * терияки бойз начинает играть*,neutral,1321540138334302209


In [24]:
test_df['new_label'] = test_df['label'].map({"positive": 0, "negative": 1, "neutral": 2, "speech": -1, "skip": -1})
test_df = test_df[test_df['new_label']>=0]
test_df.head()

,text,label,id,new_label
1,я считаю это мем года https://t.co/xoVKj5y8Mj,positive,1218052288964632576,0
2,ян русский на сотку все запятые где надо🤙🏻👍🏻👍🏻...,positive,1212859589592539136,0
4,@daria_karapet * терияки бойз начинает играть*,neutral,1321540138334302209,2
5,(пушка на Караульной горе больше не стреляет Б...,negative,1342696727808274432,1
6,@Iori_loves_U Как мило /смутилась/ спасибо 🥰🌸,positive,1317052132382679041,0


In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

#checkpoint = './teacher_model_weights/teacher_model_weights/'
tokenizer = "DeepPavlov/rubert-base-cased"
ckpt = './models/bert_teacher/'
tokenizer = AutoTokenizer.from_pretrained(tokenizer)
model = BertForSequenceClassification.from_pretrained(ckpt, num_labels=3).to(device)

cuda


In [26]:
model.dtype

torch.float32

In [27]:
print("Base Model: ", model.num_parameters())

Base Model:  177855747


In [28]:
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

id2label = pipe.model.config.id2label
label2id = pipe.model.config.label2id

Device set to use cuda:0


In [21]:
start_time = time.time()
pipe(test_df['text'].to_list())
stop_time = time.time()
print('Время: ', stop_time - start_time, ' c')
print('Время на 1 текст:', (stop_time - start_time)/len(test_df), ' c')

Время:  30.742342472076416  c
Время на 1 текст: 0.013904270679365182  c


In [29]:
from sklearn.metrics import accuracy_score, f1_score

def eval_metrics(model, df, limit=None):
    preds = model(df['text'].tolist()[:limit])
    preds = [label2id[i['label']] for i in preds]

    print("Accuracy:", accuracy_score(df['new_label'].iloc[:limit], preds))

In [30]:
eval_metrics(pipe, test_df, limit=10000)

Accuracy: 0.52962460425147


# Конвертация

In [3]:
from optimum.pipelines import pipeline
from optimum.onnxruntime import ORTOptimizer, ORTModelForSequenceClassification
from optimum.onnxruntime.configuration import OptimizationConfig

c:\Users\nikit\Desktop\Учеба\ИТМО\Генеративки\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
import torch

device = 'cuda' if torch.cuda.is_available() else "cpu"
device

'cpu'

In [8]:
from transformers import AutoTokenizer
tokenizer = "DeepPavlov/rubert-base-cased"
ckpt = './models/bert_teacher/'
tokenizer = AutoTokenizer.from_pretrained(tokenizer)
model = ORTModelForSequenceClassification.from_pretrained(ckpt, export=True).to(device)

In [10]:
model.save_pretrained('./models/bert_onnx')

# Инференс onnx

In [13]:
import onnxruntime
from optimum.pipelines import pipeline

sess = onnxruntime.InferenceSession("models/bert_onnx/model.onnx", providers=['CUDAExecutionProvider'])
session_options = onnxruntime.SessionOptions()
session_options.log_severity_level = 0

In [2]:
providers = onnxruntime.get_available_providers()
print(providers)

['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']


### Using optimum pipeline

In [4]:
optimum_model1 = pipeline(
        task="text-classification", 
        model="models/bert_onnx/", 
        tokenizer="DeepPavlov/rubert-base-cased", 
        accelerator="ort",
        device=0,
)

Device set to use cuda:0


In [18]:
id2label = optimum_model1.model.config.id2label
label2id = optimum_model1.model.config.label2id

In [9]:
sample_input = test_df['text'].to_list()

In [11]:
import time

#WARMUP7
for _ in range(10):
  _ = optimum_model1(sample_input[:10])


#INFERENCE
start = time.time()
for _ in range(10):
  _ = optimum_model1(sample_input)
total_time_model = time.time() - start
total_time_model /= 10
print(f"Общее время обработки {len(sample_input)} запросов базовой моделью из optimum:", total_time_model)
print('Среднее время на один запрос: ', total_time_model/len(sample_input))

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Общее время обработки 2211 запросов базовой моделью из optimum: 2.3005516028404234
Среднее время на один запрос:  0.0010405027602172878


In [21]:
eval_metrics(optimum_model1, test_df)

Accuracy: 0.5300768882858435
